In [1]:
import tensorflow as tf
import time,numpy
from sys import getsizeof

In [8]:
A = numpy.random.random([4001,4001])
#print(A)
mem = getsizeof(A)
print("Memory allocated: {:.3f} Megabytes".format(mem/1024/1024))

Memory allocated: 122.131 Megabytes


## Svd

### Numpy, cpu

In [9]:
start = time.time()
u,s,v = numpy.linalg.svd(A)
print("Temps d'exécution: {:.3f} secondes".format(time.time()-start))
print(s[:5])

Temps d'exécution: 38.511 secondes
[ 2000.96558648    36.58954407    36.52638443    36.31626498    36.30581044]


### Tensorflow, cpu

In [4]:
sess = tf.InteractiveSession()
sess.as_default()

with tf.device('/cpu:0'):
    s, v, u = tf.svd(A)

start = time.time()
for i in range(1):
    res = s.eval()
    #res = sess.run(s)
print("Temps d'exécution: {:.3f} secondes".format(time.time()-start))
print(res[:5])

Temps d'exécution: 2.631 secondes
[ 500.9588133    18.07021236   18.03339389   18.01782206   17.82454317]


### Tensorflow, gpu

In [5]:
sess = tf.InteractiveSession()
sess.as_default()

with tf.device('/gpu:0'):
    s, v, u = tf.svd(A)

start = time.time()
for i in range(1):
    res = s.eval()
    #res = sess.run(s)
print("Temps d'exécution: {:.3f} secondes".format(time.time()-start))
print(res[:5])

Temps d'exécution: 2.607 secondes
[ 500.9588133    18.07021236   18.03339389   18.01782206   17.82454317]


## Multiplication de matrices

### Numpy, cpu

In [3]:
start = time.time()
for i in range(1):
    B = numpy.dot(A,A)
print("Temps d'exécution: {:.3f} secondes".format(time.time()-start))
print(B[:3,:3])

Temps d'exécution: 0.394 secondes
[[ 495.7560973   505.31396242  484.8570708 ]
 [ 513.00679369  502.82144201  493.17051005]
 [ 499.11819616  498.39695839  488.03615586]]


### Tensorflow, cpu

In [9]:
sess = tf.InteractiveSession()
sess.as_default()

with tf.device('/cpu:0'):
    B = tf.matmul(A,A)

start = time.time()
for i in range(1):
    res = sess.run(B)
print("Temps d'exécution: {:.3f} secondes".format(time.time()-start))

devices = sess.list_devices()
for d in devices:
    print(d.name)
sess.close()

Temps d'exécution: 2.635 secondes
/job:localhost/replica:0/task:0/device:CPU:0
/job:localhost/replica:0/task:0/device:GPU:0


### Tensorflow, gpu

In [8]:
sess = tf.InteractiveSession()
sess.as_default()

with tf.device('/gpu:0'):
    B = tf.matmul(A,A)

start = time.time()
for i in range(1):
    res = sess.run(B)
print("Temps d'exécution: {:.3f} secondes".format(time.time()-start))

devices = sess.list_devices()
for d in devices:
    print(d.name)
sess.close()

Temps d'exécution: 2.522 secondes
/job:localhost/replica:0/task:0/device:CPU:0
/job:localhost/replica:0/task:0/device:GPU:0


In [16]:
sess = tf.InteractiveSession()
sess.as_default()

with tf.device('/cpu:0'):
    B_cpu = tf.matmul(A,A)
with tf.device('/gpu:0'):
    B_gpu = tf.matmul(A,A)
with tf.device('/gpu:0'):
    B_gpu2 = tf.matmul(A,A)
    
start = time.time()
for i in range(1):
    res_gpu2 = sess.run(B_gpu2)
print("Temps d'exécution (gpu): {:.3f} secondes".format(time.time()-start))

start = time.time()
for i in range(1):
    res_cpu = sess.run(B_cpu)
print("Temps d'exécution (cpu): {:.3f} secondes".format(time.time()-start))

start = time.time()
for i in range(1):
    res_gpu = sess.run(B_gpu)
print("Temps d'exécution (gpu): {:.3f} secondes".format(time.time()-start))

devices = sess.list_devices()
for d in devices:
    print(d.name)
sess.close()

Temps d'exécution (gpu): 1.144 secondes
Temps d'exécution (cpu): 0.514 secondes
Temps d'exécution (gpu): 0.243 secondes
/job:localhost/replica:0/task:0/device:CPU:0
/job:localhost/replica:0/task:0/device:GPU:0


In [3]:
import numpy as np
import tensorflow as tf
import time

n = 4001
X = tf.constant(np.array(np.random.randn(n,n), dtype = np.float32), dtype = tf.float32)
Y = tf.matmul(X, tf.transpose(X))

init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

a = time.time()
sess.run(Y)
print (time.time() - a)
sess.close()

Instructions for updating:
Use `tf.global_variables_initializer` instead.
4.624542713165283


In [3]:
import numpy as np
import tensorflow as tf
import time

size = 8001
x = np.array(np.random.randn(size, size), dtype = np.float32)

X = tf.placeholder(tf.float32, shape=(size, size), name=None)
Y = tf.matmul(X, X)

sess = tf.Session()

a = time.time()
sess.run(Y, feed_dict={X: x})
print (time.time() - a)

3.6510424613952637


In [4]:
import numpy as np
import time
np.random.seed(1)
n = 8001
x = np.array(np.random.randn(n,n), dtype = np.float32)
a = time.time(); x.dot(x); print (time.time() - a)

9.242243528366089


In [13]:
import operateurs_t

def U(y,t):
    return 1-y**2

Re = 10000
alpha = 0.6
N = 7
t = 30

# tensorflow

[a,b] = operateurs_t.buildAB_forward(Re, alpha, N, U, t)
A = tf.placeholder(tf.complex128, shape=(N-4, N-4))
B = tf.placeholder(tf.complex128, shape=(N-4, N-4))

with tf.device('/gpu:0'):
    B_inv = tf.matrix_inverse(B)
    M = tf.multiply(-1j*alpha,tf.matmul(B_inv,A))

sess = tf.Session()
numpyM = sess.run(M, feed_dict={A: a, B: b})

print(numpyM)

# numpy

[A,B] = operateurs_t.buildAB_forward(Re, alpha, N, U, t)
B_inv = numpy.linalg.inv(B)
M = -1j*alpha*numpy.dot(B_inv,A)

print(M)

[[-0.00236777-0.37338009j  0.00117791-0.0016146j  -0.00028540+0.08546534j]
 [ 0.00038913+0.15298297j -0.00156354-0.53622346j  0.00038913+0.15298297j]
 [-0.00028540+0.08546534j  0.00117791-0.0016146j  -0.00236777-0.37338009j]]
[[-0.00236777-0.37338009j  0.00117791-0.0016146j  -0.00028540+0.08546534j]
 [ 0.00038913+0.15298297j -0.00156354-0.53622346j  0.00038913+0.15298297j]
 [-0.00028540+0.08546534j  0.00117791-0.0016146j  -0.00236777-0.37338009j]]


In [38]:
import operateurs_t

def U(y,t):
    return 1-y**2

Re = 10000
alpha = 0.6
N = 7
t = 30

# tensorflow
def f(t,L,Re,alpha,N,U):
    
    [A,B] = operateurs_t.buildAB_forward(Re, alpha, N, U, t)
    B = B*(1+0.0j)

    with tf.device('/gpu:0'):
        B_inv = tf.matrix_inverse(B)
        M = tf.multiply(-1j*alpha,tf.matmul(B_inv,A))
        L_new = tf.matmul(L,M)
    
    sess = tf.Session()
    L_new_n = sess.run(L_new)
    
    return L_new_n

L = numpy.eye(N-4)*(1+0.0j)
print(f(t,L,Re,alpha,N,U))

# numpy

[A,B] = operateurs_t.buildAB_forward(Re, alpha, N, U, t)
B_inv = numpy.linalg.inv(B)
M = -1j*alpha*numpy.dot(B_inv,A)

print(M)

[[-0.00236777-0.37338009j  0.00117791-0.0016146j  -0.00028540+0.08546534j]
 [ 0.00038913+0.15298297j -0.00156354-0.53622346j  0.00038913+0.15298297j]
 [-0.00028540+0.08546534j  0.00117791-0.0016146j  -0.00236777-0.37338009j]]
[[-0.00236777-0.37338009j  0.00117791-0.0016146j  -0.00028540+0.08546534j]
 [ 0.00038913+0.15298297j -0.00156354-0.53622346j  0.00038913+0.15298297j]
 [-0.00028540+0.08546534j  0.00117791-0.0016146j  -0.00236777-0.37338009j]]


In [4]:
a = 2*numpy.eye(2)
b = numpy.eye(2)
b[1,0] = 1
cT = tf.matmul(a,b)
sess = tf.Session()
c = sess.run(cT)
print(c)

[[ 2.  0.]
 [ 2.  2.]]


In [ ]:
import operateurs_t,scipy,numpy
import tensorflow as tf

def U(y,t):
    return 1-y**2

Re = 10000
alpha = 0.6
N = 41
t_max = 0.1
dt = 0.1
step = 20

st,t_vec = svd_tf_RK4(Re,alpha,N,t_max,dt,step,U)
print(st)

In [1]:
def f(t,L,Re,alpha,N,U):
    
    [A,B] = operateurs_t.buildAB_forward(Re, alpha, N, U, t)
    B = B*(1+0.0j)

    with tf.device('/gpu:0'):
        B_inv = tf.matrix_inverse(B)
        M = tf.multiply(-1j*alpha,tf.matmul(B_inv,A))
        L_new = tf.matmul(L,M)
    
    sess = tf.Session()
    sess.run(L_new)
    
    return L_new

def svd_tf_RK4(Re,alpha,N,t_max,dt,step,U):

    nt = int(t_max/dt)+1
    st = numpy.empty((int(nt/step)+1,N-4)) # valeurs singulières
    t_vec = numpy.empty((int(nt/step)+1,1))

    t = 0
    L = scipy.eye(N-4)*(1+0.0j)
    t_vec[0] = 0
    st[0,:] = 1
    
    sess = tf.Session()

    for i in range(1,nt+1):
    
        # La matrice L=exp(Mt) est avancé de dt dans le temps avec RK4
        with tf.device('/gpu:0'):
            k1 = f(t,L,Re,alpha,N,U)
            k2 = f(t+dt/2,tf.add(L,tf.multiply(k1,1/2*(1+0.0j))),Re,alpha,N,U)
            k3 = f(t+dt/2,tf.add(L,tf.multiply(k2,1/2*(1+0.0j))),Re,alpha,N,U)
            k4 = f(t+dt,tf.add(L,k3),Re,alpha,N,U)
    
            # L = L + dt/6*(k1+2*k2+2*k3+k4)
            temp = tf.add(k1,tf.add(tf.multiply(2+0.0j,k2),tf.add(tf.multiply(2+0.0j,k3),k4)))
            L = tf.add(L,tf.multiply(dt/6*(1+0.0j),temp))
    
        t =t+dt
        sess.run(L)
        
        # Calcul de svd
        if i%step==0:
            with tf.device('/cpu:0'):
                s = tf.svd(L,compute_uv=False)
            
            index = int(i/step)
            s_numpy = sess.run(s)
            st[index,:] = s_numpy
            t_vec[index] = t
            
    return st,t_vec

### Step by step w/ tensorflow

In [2]:
import operateurs_t,scipy,numpy
import tensorflow as tf

def U(y,t):
    return 1-y**2

In [6]:
Re = 10000
alpha = 0.6
N = 401
t_max = 0.1
dt = 0.1
step = 20

t = 0.1
L = scipy.eye(N-4)*(1+0.0j)

[a,b] = operateurs_t.buildAB_forward(Re, alpha, N, U, t)
b = b*(1+0.0j)
A = tf.placeholder(tf.complex128, shape=(N-4, N-4))
B = tf.placeholder(tf.complex128, shape=(N-4, N-4))

with tf.device('/gpu:0'):
    B_inv = tf.matrix_inverse(B)
    M = tf.multiply(-1j*alpha,tf.matmul(B_inv,A))
    L_new = tf.matmul(L,M)
    
sess = tf.Session()
res = sess.run(B_inv,feed_dict={A:a, B:b})
print(res)

[[ -3.31734923e-05-0.j  -3.30539101e-05-0.j  -3.29346253e-05-0.j ...,
   -2.21896476e-07-0.j  -1.55326235e-07-0.j  -8.87573923e-08-0.j]
 [ -3.30539101e-05-0.j  -5.78446401e-05-0.j  -5.76358907e-05-0.j ...,
   -3.88320830e-07-0.j  -2.71822310e-07-0.j  -1.55326235e-07-0.j]
 [ -3.29346253e-05-0.j  -5.76358907e-05-0.j  -8.23376748e-05-0.j ...,
   -5.54748680e-07-0.j  -3.88320830e-07-0.j  -2.21896476e-07-0.j]
 ..., 
 [ -2.21896476e-07-0.j  -3.88320830e-07-0.j  -5.54748680e-07-0.j ...,
   -8.23376748e-05-0.j  -5.76358907e-05-0.j  -3.29346253e-05-0.j]
 [ -1.55326235e-07-0.j  -2.71822310e-07-0.j  -3.88320830e-07-0.j ...,
   -5.76358907e-05-0.j  -5.78446401e-05-0.j  -3.30539101e-05-0.j]
 [ -8.87573923e-08-0.j  -1.55326235e-07-0.j  -2.21896476e-07-0.j ...,
   -3.29346253e-05-0.j  -3.30539101e-05-0.j  -3.31734923e-05-0.j]]


In [1]:
import operateurs_t,scipy,numpy
import tensorflow as tf

def U(y,t):
    return 1-y**2

Re = 10000
alpha = 0.6
N = 401
t = 0.1

tf.InteractiveSession()

[a,b] = operateurs_t.buildAB_forward(Re, alpha, N, U, t)
b = b *(1+0.0j)

B = tf.placeholder(tf.complex128, shape=(N-4, N-4))
#with tf.device('/gpu:0'):
B_inv = tf.matrix_inverse(B).eval(feed_dict={B: b})

In [14]:
import numpy
import tensorflow as tf

sess = tf.Session()

N = 401

a = numpy.ones((N,N))
A = tf.placeholder(tf.float64, shape=(N, N))
B = A

for i in range(1000):
    B = tf.add(B,A)
    
sess.run(B, feed_dict={A:a})

array([[ 1001.,  1001.,  1001., ...,  1001.,  1001.,  1001.],
       [ 1001.,  1001.,  1001., ...,  1001.,  1001.,  1001.],
       [ 1001.,  1001.,  1001., ...,  1001.,  1001.,  1001.],
       ..., 
       [ 1001.,  1001.,  1001., ...,  1001.,  1001.,  1001.],
       [ 1001.,  1001.,  1001., ...,  1001.,  1001.,  1001.],
       [ 1001.,  1001.,  1001., ...,  1001.,  1001.,  1001.]])

### Tensorflow learning examples

In [1]:
import numpy,scipy
import tensorflow as tf
import operateurs_t

In [2]:
def f(t,L,Re,alpha,N,U):
        
    [a,b] = operateurs_t.buildAB_forward(Re, alpha, N, U, t)
    b = b*(1+0.0j)
    
    A = tf.constant(a)
    B = tf.constant(b)

    with tf.device('/gpu:0'):
        B_inv = tf.matrix_inverse(B)
        M = tf.multiply(-1j*alpha,tf.matmul(B_inv,A))
        L_new = tf.matmul(L,M)
    
    return L_new

In [3]:
def U(y,t):
    return 1-y**2

Re = 10000
alpha = 0.6
N = 20

t = 0.1
L = tf.constant(scipy.eye(N-4)*(1+0.0j))

##############################
sess = tf.Session()

res = f(t,L,Re,alpha,N,U)
print(sess.run(res))

sess.close()
##############################

[[ -2.59798293e-02-0.10731886j   1.17316878e-02-0.01615346j
    4.59385746e-18-0.01305278j  -2.11617397e-18-0.01021932j
   -7.57187616e-18-0.00762327j   1.53706411e-18-0.00523659j
    5.69046347e-18-0.00303277j  -8.03205868e-18-0.00098667j
   -6.04246893e-18+0.00092561j   1.60157849e-17+0.00272695j
   -1.54983096e-17+0.00443932j   5.85490427e-18+0.00608393j
   -1.49469809e-18+0.0076814j    7.52783208e-19+0.00925184j
    1.86430763e-04+0.01081505j  -6.99115362e-04+0.01458796j]
 [ -3.17189117e-04+0.19493702j  -1.55947496e-02-0.25437177j
    9.02500000e-03-0.02289444j  -9.28855077e-18-0.01792457j
   -6.15669495e-18-0.01337113j  -6.83887059e-18-0.00918492j
    2.28422758e-17-0.00531944j  -2.17614187e-17-0.00173062j
   -5.21754408e-18+0.00162351j   2.11256278e-17+0.00478304j
   -1.55442977e-17+0.00778652j   3.44729569e-18+0.01067115j
   -1.10599982e-18+0.01347308j  -1.24392619e-18+0.01622762j
    3.26997493e-04+0.01896949j  -1.22624060e-03+0.02558712j]
 [ -8.57156418e-03+0.17885692j   1.131

In [18]:
import numpy,scipy,time
import tensorflow as tf
import operateurs_t
#############################
def f(N):
    B = tf.constant(scipy.eye(N))
    with tf.device('/gpu:0'):
        #B_inv = tf.matrix_inverse(B)
        B_inv = tf.matmul(B,B)
        B_inv = tf.matmul(B_inv,B)
    return B_inv
##############################
N = 2000
##############################
sess = tf.Session()
res = f(N)
start = time.time()
print(len(sess.run(res)))
print("Exécution en secondes =", time.time()-start)
sess.close()
##############################

2000
Exécution en secondes = 3.263012647628784


In [16]:
import numpy,scipy,time
##############################
def f(N):
    b = scipy.eye(N)
    return numpy.dot(b,b)
##############################
N = 2000
##############################
start = time.time()
print(len(f(N)))
print("Exécution en secondes =", time.time()-start)

2000
Exécution en secondes = 0.2514970302581787
